In [1]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt




In [2]:
# Choose dataset 1 or 2:
dataset = 1  # 1 or 2

if dataset == 1:
    data_folder = "1_train-val_1min_aalesund_from_start/"
    yaml_file_combined = "yaml_files/data_1.yaml"
    yaml_file_ball = "yaml_files/ball_1.yaml"
    yaml_file_players = "yaml_files/player_1.yaml"

else:
    data_folder = "2_train-val_1min_after_goal/"
    yaml_file_combined = "yaml_files/data_2.yaml"
    yaml_file_ball = "yaml_files/ball_2.yaml"
    yaml_file_players = "yaml_files/player_2.yaml"

test_combined = "yaml_files/test_files/test_combined.yaml"
test_ball = "yaml_files/test_files/test_ball.yaml"
test_players = "yaml_files/test_files/test_players.yaml"

img_width = 1920  
img_height = 1080  

In [3]:
# Generate shuffled txt and img files for training and validation, 80% training and 20% validation

from scripts.shuffler import generate_shuffled_txt_img_files_for_train_val
from scripts.ball_dataset import generate_ball_dataset
from scripts.player_dataset import generate_player_dataset

object_path = f"data_yolov8/object_datasets/{data_folder}/"  
ball_path = f"data_yolov8/ball_datasets/{data_folder}/"  
player_path = f"data_yolov8/player_datasets/{data_folder}/"
text_file_path = f"/datasets/tdt4265/other/rbk/{data_folder}/gt/gt.txt"  
all_images_path = f"/datasets/tdt4265/other/rbk/{data_folder}/img1"



generate_shuffled_txt_img_files_for_train_val(text_file_path, img_width, img_height, object_path, all_images_path)
generate_ball_dataset(text_file_path, img_width, img_height, ball_path, all_images_path)
generate_player_dataset(text_file_path, img_width, img_height, player_path, all_images_path)


In [4]:
# Generate test txt and img files

from scripts.test_dataset import generate_txt_img_files_for_test

object_path = f"data_yolov8/object_datasets/3_test_1min_hamkam_from_start/"  
ball_path = f"data_yolov8/ball_datasets/3_test_1min_hamkam_from_start/"  
player_path = f"data_yolov8/player_datasets/3_test_1min_hamkam_from_start/" 
text_file_path = "/datasets/tdt4265/other/rbk/3_test_1min_hamkam_from_start/gt/gt.txt"  
all_images_path = "/datasets/tdt4265/other/rbk/3_test_1min_hamkam_from_start/img1/"

generate_txt_img_files_for_test(text_file_path, img_width, img_height, object_path, all_images_path)
generate_txt_img_files_for_test(text_file_path, img_width, img_height, ball_path, all_images_path, type='ball')
generate_txt_img_files_for_test(text_file_path, img_width, img_height, player_path, all_images_path, type='player')

In [6]:
# Last inn sist trente modell

def find_latest_model_with_best(base_path, type):
    detect_path = os.path.join(base_path, 'runs/detect', type)
    training_sessions = [os.path.join(detect_path, d) for d in os.listdir(detect_path) if os.path.isdir(os.path.join(detect_path, d))]
    
    # Liste for å holde stiene til alle 'best.pt'-filer
    best_model_paths = []

    for session in training_sessions:
        best_model_path = os.path.join(session, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            best_model_paths.append((best_model_path, os.path.getmtime(best_model_path)))

    if not best_model_paths:
        print("Ingen 'best.pt' fil funnet i noen av treningsøktene.")
        return None
    
    # Sorter basert på modifiseringstid og returner den nyeste
    best_model_paths.sort(key=lambda x: x[1], reverse=True)
    return best_model_paths[0][0]


base_path = '/work/mbergst/TDT4265_Project' 

#model_combined = YOLO(find_latest_model_with_best(base_path, 'combined'))
model_ball = YOLO(find_latest_model_with_best(base_path, 'ball'))
model_players = YOLO(find_latest_model_with_best(base_path, 'player'))
#model = YOLO('yolov8n.pt')


In [5]:
# Object Detection

model_combined = YOLO('yolov8n.pt')
model_ball = YOLO('yolov8n.pt')
model_players = YOLO('yolov8n.pt')

# Tren modellen på datasettet
results = model_combined.train(data=yaml_file_combined, epochs=50, batch=14, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect/combined')
#results = model_ball.train(data=yaml_file_ball, epochs=50, batch=14, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect/ball')
#results = model_players.train(data=yaml_file_players, epochs=50, batch=14, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect/player')




New https://pypi.org/project/ultralytics/8.2.5 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yaml_files/data_1.yaml, epochs=50, time=None, patience=100, batch=14, imgsz=(1920, 1080), save=True, save_period=-1, cache=False, device=None, workers=8, project=/work/mbergst/TDT4265_Project/runs/detect/combined, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False

train: Scanning /work/mbergst/TDT4265_Project/data_yolov8/object_datasets/1_train-val_1min_aalesund_from_start/labels/train... 1441 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1441/1441 [00:00<00:00, 2463.73it/s]


train: New cache created: /work/mbergst/TDT4265_Project/data_yolov8/object_datasets/1_train-val_1min_aalesund_from_start/labels/train.cache


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/object_datasets/1_train-val_1min_aalesund_from_start/labels/val... 361 images, 0 backgrounds, 0 corrupt: 100%|██████████| 361/361 [00:00<00:00, 4614.20it/s]

val: New cache created: /work/mbergst/TDT4265_Project/data_yolov8/object_datasets/1_train-val_1min_aalesund_from_start/labels/val.cache


...

                   all        361       8361      0.963      0.906      0.934       0.75



50 epochs completed in 0.400 hours.
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/detect/combined/train11/weights/last.pt, 6.5MB
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/detect/combined/train11/weights/best.pt, 6.5MB

Validating /work/mbergst/TDT4265_Project/runs/detect/combined/train11/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  3.37it/s]


                   all        361       8361      0.962       0.91      0.936      0.753
                  ball        361        241      0.927      0.822      0.877      0.584
                player        361       8120      0.996      0.999      0.995      0.922
Speed: 0.3ms preprocess, 1.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /work/mbergst/TDT4265_Project/runs/detect/combined/train11


NameError: name 'r' is not defined

In [5]:
model_combined = YOLO("/work/mbergst/TDT4265_Project/runs/detect/combined/train7/weights/best.pt")
#model_combined.predict('/work/mbergst/TDT4265_Project/data_yolov8/object_datasets/1_train-val_1min_aalesund_from_start/images/all_frames/000763.jpg', 
                       #save=True, imgsz=(1920, 1080), conf=0.5, project='/work/mbergst/TDT4265_Project/runs/detect/combined')

In [22]:
# Evaluér modellens prestasjon på valideringssettet
# Gjør det litt ulikt train sitt output pga ingen parametere definert
#val_results = model_combined.val()


In [ ]:
# Test modellen på testsette

test_ball = model_ball.val(data=test_ball, cfg='test_hyperparameters.yaml',batch=10, imgsz=(1920, 1080), project = '/work/mbergst/TDT4265_Project/runs/detect/ball/test')
#test_players = model_players.val(data=test_players, batch=8, imgsz=(1920, 1080), project = '/work/mbergst/TDT4265_Project/runs/detect/player/test')


In [3]:
# Object tracking
all_frames = f'/datasets/tdt4265/other/rbk/{data_folder}/img1'

frame_paths = sorted([os.path.join(all_frames, f) for f in os.listdir(all_frames) if f.endswith('.jpg')])


In [ ]:
# Iterere over hver frame og bruk track metoden
for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model_combined.track(frame_path, persist=True, stream=True, line_width=1, tracker='botsort.yaml'):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        # Vent på et tastetrykk i 1 millisekund og sjekk om det er 'ESC'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # 27 er ASCII-verdien for 'ESC'
            break

    if key == 27:  # Kontroller igjen om 'ESC' ble trykket for å bryte ut av den ytre løkken
        break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene

In [4]:
# Blender frames fra 2 modeller
model_ball = YOLO('/work/mbergst/TDT4265_Project/runs/detect/ball/train44/weights/best.pt')
model_players = YOLO('/work/mbergst/TDT4265_Project/runs/detect/player/train6/weights/best.pt')

for frame_path in frame_paths:
    # Hent resultatene fra begge modellene for hver frame
    for result1, result2 in zip(model_players.track(frame_path, persist=True, stream=True, line_width=1, tracker = 'botsort.yaml'),
                                model_ball.track(frame_path, persist=True, stream=True, line_width=1, tracker = 'botsort.yaml')):
        # Tegn annotasjoner fra begge modellene på samme bilde
        annotated_frame1 = result1.plot(font_size=1, line_width=1)
        annotated_frame2 = result2.plot(font_size=1, line_width=1)

        combined_frame = cv2.addWeighted(annotated_frame1, 0.5, annotated_frame2, 0.5, 0)  

        # Vis det annoterte bildet med resultater fra begge modeller
        cv2.imshow('Combined frame', combined_frame)
        
        # Vent på et tastetrykk i 1 millisekund og sjekk om det er 'ESC'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # 27 er ASCII-verdien for 'ESC'
            break

    if key == 27:  # Kontroller igjen om 'ESC' ble trykket for å bryte ut av den ytre løkken
        break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene



image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000001.jpg: 1088x1920 22 players, 55.7ms

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000001.jpg: 1088x1920 1 ball, 11.5ms
Speed: 5.6ms preprocess, 55.7ms inference, 474.6ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000002.jpg: 1088x1920 22 players, 4.2ms

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000002.jpg: 1088x1920 1 ball, 4.3ms
Speed: 4.8ms preprocess, 4.2ms inference, 12.8ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000003.jpg: 1088x1920 21 players, 4.5ms

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000003.jpg: 1088x1920 1 ball, 4.5ms
Speed: 6.5ms preprocess, 4.5ms inference, 12.3ms postprocess per image at shape (1, 

In [26]:
# Object tracking with supervision

boxann = sv.BoxAnnotator(
    thickness=1,
    text_thickness=1,
    text_scale=0.5,
    
)

for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_thickness=1):
        frame = result.orig_img
        
        detections = sv.Detections.from_ultralytics(result)
        

        frame = boxann.annotate(scene=frame, detections=detections)
        cv2.imshow('yolov8', frame)
        cv2.waitKey(1)  # Venter en kort tid så bildet kan oppdateres

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene

WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'line_width' instead.

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 22 players, 4.5ms


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
# Not working Object tracking

for frame in frame_paths:
    results = model.track(frame, persist=True)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    ids = results[0].boxes.id.cpu().numpy().astype(int)
    for box, id in zip(boxes, ids):
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"Id {id}",
            (box[0], box[1]),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
        )
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break


image 1/1 /work/mbergst/project/data_yolov8/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 2 players, 3.2ms
Speed: 21.6ms preprocess, 3.2ms inference, 0.6ms postprocess per image at shape (1, 3, 1088, 1920)


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
